In [1]:
import numpy as np
import pandas as pd

import arviz as az

import bebi103

import bokeh.io
import bokeh.plotting
bokeh.io.output_notebook()

/home/ec2-user/miniconda/lib/python3.7/site-packages/bebi103/viz.py:30: UserWarning: DataShader import failed with error "No module named 'datashader'".
Features requiring DataShader will not work and you will get exceptions.
  Features requiring DataShader will not work and you will get exceptions.""")


Loading BokehJS ...

## Problem 8.2: Microtubule catastrophe, 40 pts

_Note: This problem is best done after the lecture November 22._

In this problem, we use data from [Gardner, Zanic, et al., Depolymerizing kinesins Kip3 and MCAK shape cellular microtubule architecture by differential control of catastrophe, *Cell*, **147**, 1092-1103, 2011](https://doi.org/10.1016/j.cell.2011.10.037). The authors investigated the dynamics of microtubule catastrophe, the switching of a microtubule from a growing to a shrinking state.  In particular, they were interested in the time between the start of growth of a microtubule and the catastrophe event. They monitored microtubules in a single-molecule [TIRF assay](https://en.wikipedia.org/wiki/Total_internal_reflection_fluorescence_microscope) by using tubulin (the monomer that comprises a microtubule) that was labeled with a fluorescent marker. As a control to make sure that fluorescent labels and exposure to laser light did not affect the microtubule dynamics, they performed a similar experiment using differential interference contrast (DIC) microscopy. They measured the time until catastrophe with labeled and unlabeled tubulin. We will carefully analyze the data and make some conclusions about the processes underlying microtubule catastrophe.

In the file `gardner_mt_catastrophe_only_tubulin.csv` (which you can download [here](../data/gardner_mt_catastrophe_only_tubulin.csv)), we have observed catastrophe times of microtubules with different concentrations of tubulin. To start with, we will consider the experiment run with a tubulin concentration of 12 µM. So, our data set consists of a set of measurements of the amount of time to catastrophe. We will consider three models for microtubule catastrophe.

- Model 1: The time to catastrophe is Exponentially distributed.
- Model 2: The time to catastrophe is Gamma distributed.
- Model 3: The time to catastrophe is Weibull distributed.

Note that these descriptions are for the likelihood; we have not specified priors.


**a)**  Describe the three models in words. Give physical descriptions of the meanings of their parameters. Describe how these models are related to each other. Tutorial 3c will be useful. 

<br />



- Model 1: The time to catastrophe is Exponentially distributed.

This suggests that the occurance of catastrophe is a Poisson process, so it is a "rare event" that requires multiple subprocesses to lead it it. The parameter for the process, if it is exponential, $\beta$, represents the characteristic rate of catastrophe, that is how often catastrophe happens in a certain amount of time. It can also be parametrized as $\tau=1/\beta$, the characteristic catastrophe time, which fits what we are given in our data. The Exponential distribution is a special case of the Gamma distribution where $\alpha = 1$ and a special case of the Weibull distribution where $\alpha = 1$ and $\sigma=1/\beta$



- Model 2: The time to catastrophe is Gamma distributed.

This suggests that the occurance of catastrophe represents a specific number of occurances of a Poisson process, that is a discrete number of steps that occur at the same rate must occur for catastrophe to occur. There are two parameters for this distribution, $\alpha$ and $\beta$, where $\alpha$ is the number of arrivals (or "steps") required to trigger catastrophe, and $\beta$ is the rate of the arrivals. Thus, the characteristic catastrophe time is given by $\alpha/\beta$. 


- Model 3: The time to catastrophe is Weibull distributed.

This suggests that the likelihood of catastrophe is dependent on the amount of time it has been since the last catastrophe, so the longer it has been since the last catastrophe, the more likely it is that catastrophe will occur. There are two parameters for this distribution, $\alpha$ which defines how the probability changes over time, and $\sigma$ which is the characteristic catastrophe time.

**b)** Perform parameter estimates for the respective models and make model comparisons. Comment on what this means with respect to our understanding of how microtubule catastrophe works.

<br />




**c)** Using whichever model you favor based on your work in part (b), obtain parameter estimates for the other tubulin concentrations. Given that microtubules polymerize faster with higher tubulin concentrations, is there anything you can say about the occurrence of catastrophe by looking at the values of the parameters versus tubulin concentration?


In [2]:
df = pd.read_csv('/home/ec2-user/data/gardner_mt_catastrophe_only_tubulin.csv', comment = "#")

In [3]:
df.tail()

,12 uM,7 uM,9 uM,10 uM,14 uM
687,1335.0,NaN,NaN,NaN,NaN
688,1485.0,NaN,NaN,NaN,NaN
689,1505.0,NaN,NaN,NaN,NaN
690,1520.0,NaN,NaN,NaN,NaN
691,1785.0,NaN,NaN,NaN,NaN


In [4]:
prior_sm1 = bebi103.stan.StanModel(file='./8.2_prior_pred_12_m1.stan')

Using cached StanModel.


In [5]:
# Store input parameters in a dictionary so stan can access them
data = dict(N=692)

# Generate samples
samples_gen1 = prior_sm1.sampling(data=data,
                          algorithm='Fixed_param',
                          warmup=0,
                          chains=1,
                          iter=1000)

# Store samples in a dataframe
df_gen1 = bebi103.stan.to_dataframe(samples_gen1, diagnostics=False)
df_gen1.head()

,chain,chain_idx,warmup,uM_12[1],uM_12[2],uM_12[3],uM_12[4],uM_12[5],uM_12[6],uM_12[7],...,uM_12[685],uM_12[686],uM_12[687],uM_12[688],uM_12[689],uM_12[690],uM_12[691],uM_12[692],beta_,lp__
0,1,1,0,38.376494,66.921094,7.883437,4.922266,25.335615,20.639976,9.751084,...,15.583129,5.374144,11.244927,22.760366,16.176543,0.627469,73.667255,14.250200,0.043860,0.0
1,1,2,0,13.330826,21.012301,13.955080,46.176481,36.157353,35.740289,16.240803,...,57.661527,7.555743,30.347809,4.314168,5.665554,20.184150,27.865459,11.207015,0.053790,0.0
2,1,3,0,53.523595,6.886408,27.230859,2.475122,18.899126,4.399795,32.204598,...,6.829320,20.092252,15.205548,14.247257,84.833635,28.450141,40.136138,55.857332,0.036587,0.0
3,1,4,0,42.260243,27.485236,2.199345,45.359842,26.095366,21.828360,40.251160,...,26.545083,5.145437,16.793327,5.996164,6.602217,57.997227,26.498576,167.133116,0.024150,0.0
4,1,5,0,16.212615,30.099423,9.932559,32.790024,3.575002,9.137695,25.094700,...,52.860296,46.139154,8.579467,3.540378,1.748560,8.782993,5.045935,6.302341,0.042152,0.0


In [21]:
p = bebi103.viz.predictive_ecdf(samples_gen1, "uM_12",
                                x_axis_label = "intercatastrophe time (s)")
p.x_range = bokeh.models.Range1d(-10, 200)
bokeh.io.show(p)

In [36]:
sm1 = bebi103.stan.StanModel(file='./8.2_mcmc_12_m1.stan')
# sm2 = bebi103.stan.StanModel(file='./8.2_mcmc_12_m2.stan')
# sm3 = bebi103.stan.StanModel(file='./8.2_mcmc_12_m3.stan')

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_848768c4c8227c0fc15a1caaa21965a4 NOW.
/home/ec2-user/miniconda/lib/python3.7/site-packages/Cython/Compiler/Main.py:367: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /tmp/tmp3gjmrhxp/stanfit4anon_model_848768c4c8227c0fc15a1caaa21965a4_6564635283508566750.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)


In [37]:
data = dict(N=len(df),
           uM_12=df['12 uM'].values.astype(float))

In [38]:
samples1 = sm1.sampling(data=data)
# samples2 = sm2.sampling(data=data)
# samples3 = sm3.sampling(data=data)

In [39]:
df_mcmc1 = bebi103.stan.to_dataframe(samples1, diagnostics=False, inc_warmup=False)
# df_mcmc2 = bebi103.stan.to_dataframe(samples2, diagnostics=False, inc_warmup=False)

In [40]:
df_mcmc1.head()

,chain,chain_idx,warmup,tao,beta_,lp__
0,1,1,0,369.450272,0.002707,-4809.375825
1,1,2,0,395.078219,0.002531,-4808.734555
2,1,3,0,384.024426,0.002604,-4808.626727
3,1,4,0,382.583508,0.002614,-4808.653702
4,1,5,0,380.461358,0.002628,-4808.711535


In [41]:
plot1 = bebi103.viz.histogram(df_mcmc2["beta_"], x_axis_label="beta_", 
                               plot_height=200, plot_width=300, 
                               bins=30, density=True) 
                 
bokeh.io.show(plot1)

In [47]:
bebi103.stan.compare({'exponential': samples1, 'gamma': samples2},
                     log_likelihood='log_like', ic='loo')

ValueError: No parameter log_like

**b)** Perform parameter estimates for the respective models and make model comparisons. Comment on what this means with respect to our understanding of how microtubule catastrophe works.

<br />

Exponential(t)

t = LogNorm(20, 10)

Gamma($\alpha$, $\beta$)

$\alpha$/$\beta$ = logNorm(20,10)

$\alpha$= Geometric(.1)

Weibull($\alpha$, $\sigma$)

$\sigma$ = LogNorm(20, 10)
$\alpha$ = LogNorm(1, 10)




In [ ]:
3, .05 alpha, 10, 3 sigma w(a, s)